In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Tutorial

In [100]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor, PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor


In [21]:
boston = load_boston()

In [22]:
X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test_split(boston.data,
                                                                                boston.target,
                                                                                test_size=0.2, 
                                                                                random_state=42)


In [23]:
robust = SGDRegressor(loss='huber',
                      penalty='l2', 
                      alpha=0.0001, 
                      fit_intercept=False, 
                      n_iter=5, 
                      shuffle=True, 
                      verbose=1, 
                      epsilon=0.1, 
                      random_state=42, 
                      learning_rate='invscaling', 
                      eta0=0.01, 
                      power_t=0.5)

model = MultiOutputRegressor(robust)

In [7]:
sc_boston = StandardScaler()
X_train_boston = sc_boston.fit_transform(X_train_boston)
X_test_boston = sc_boston.transform(X_test_boston)

In [8]:
robust.fit(X_train_boston, y_train_boston)

-- Epoch 1
Norm: 0.01, NNZs: 13, Bias: 0.000000, T: 404, Avg. loss: 2.274738
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.01, NNZs: 13, Bias: 0.000000, T: 808, Avg. loss: 2.274682
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.01, NNZs: 13, Bias: 0.000000, T: 1212, Avg. loss: 2.274675
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.01, NNZs: 13, Bias: 0.000000, T: 1616, Avg. loss: 2.274671
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.01, NNZs: 13, Bias: 0.000000, T: 2020, Avg. loss: 2.274669
Total training time: 0.01 seconds.


/Users/b1017579/.pyenv/versions/3.6.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=False, l1_ratio=0.15,
       learning_rate='invscaling', loss='huber', max_iter=None, n_iter=5,
       n_iter_no_change=5, penalty='l2', power_t=0.5, random_state=42,
       shuffle=True, tol=None, validation_fraction=0.1, verbose=1,
       warm_start=False)

In [9]:
mean_squared_error(y_test_boston, robust.predict(X_test_boston)) ** 0.5


23.13256808763112

# Demand Forecasting

In [13]:
import os.path
import sys
from datetime import date, datetime
sys.path.append(os.path.join(os.path.dirname(os.path.realpath('__file__')), '../../..'))
ROOT_DIR = os.path.join(os.path.dirname(os.path.realpath('__file__')), '')

from run.market_forecasting_comparison.munging.multi_step_forecasting_wrangling import multi_step_data_prep, get_hours_of_days_needed
# from run.market_forecasting_comparison.ML.EstimatorSelectionHelper import EstimatorSelectionHelper

import numpy as np

In [16]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
# seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 19))),
#            ('spring', (date(Y,  3, 20),  date(Y,  6, 20))),
#            ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
#            ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
#            ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

seasons = [('1', (date(Y,  4,  1),  date(Y,  4, 26))),
           ('2', (date(Y,  4, 27),  date(Y,  8, 16))),
           ('3', (date(Y,  8, 17),  date(Y,  9, 20))),
           ('4', (date(Y,  9, 21),  date(Y, 10, 25))),
           ('5', (date(Y,  10, 26),  date(Y, 12, 31))),
           ('6', (date(Y, 1, 1),  date(Y, 1, 24))),
           ('7', (date(Y, 1, 25),  date(Y, 3, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

def working_day_check(date):
    if date.holiday == True or date.dayofweek == 6:
        return "non_working_day"
    elif date.dayofweek in [0,1,2,3,4,5]:
        return "working_day"
    else:
        print("error")
    

In [17]:
demand = pd.read_csv('{}/../data/capacity/demand.csv'.format(ROOT_DIR))

demand = demand[demand.time < '2018']


prev_days_needed = get_hours_of_days_needed(days_wanted=[1, 2, 7, 30], hours_wanted=[28, 28, 28, 28, 28])
multi_step_dat = multi_step_data_prep(dat=demand, input_lags=prev_days_needed, outputs=24)
multi_step_dat.time = pd.to_datetime(multi_step_dat.time)
multi_step_dat['season'] = multi_step_dat.time.map(lambda x: get_season(x))

multi_step_dat['working_day'] = multi_step_dat.apply(lambda x: working_day_check(x), axis=1)

multi_step_dat = multi_step_dat.drop(columns=['time', 'Unnamed: 0'])

multi_step_dat


,value,hour,month,dayofweek,dayofmonth,year,holiday,value-1,value-2,value-3,...,n-740,n-741,n-742,n-743,n-744,n-745,n-746,n-747,season,working_day
770,34214.583333,21,7,1,5,2011,False,35869.833333,37017.666667,38274.583333,...,25302.750000,26183.750000,26950.916667,27883.666667,30041.916667,33132.083333,33181.000000,33880.250000,2,working_day
771,30264.500000,22,7,1,5,2011,False,34214.583333,35869.833333,37017.666667,...,24665.833333,25302.750000,26183.750000,26950.916667,27883.666667,30041.916667,33132.083333,33181.000000,2,working_day
772,28211.333333,23,7,1,5,2011,False,30264.500000,34214.583333,35869.833333,...,24041.250000,24665.833333,25302.750000,26183.750000,26950.916667,27883.666667,30041.916667,33132.083333,2,working_day
773,27015.750000,0,7,2,6,2011,False,28211.333333,30264.500000,34214.583333,...,25311.166667,24041.250000,24665.833333,25302.750000,26183.750000,26950.916667,27883.666667,30041.916667,2,working_day
774,26275.750000,1,7,2,6,2011,False,27015.750000,28211.333333,30264.500000,...,27123.500000,25311.166667,24041.250000,24665.833333,25302.750000,26183.750000,26950.916667,27883.666667,2,working_day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57294,35223.666667,19,12,6,31,2017,False,37531.583333,38381.416667,36561.083333,...,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,48157.916667,49271.000000,49574.833333,5,non_working_day
57295,32496.500000,20,12,6,31,2017,False,35223.666667,37531.583333,38381.416667,...,32920.250000,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,48157.916667,49271.000000,5,non_working_day
57296,29994.250000,21,12,6,31,2017,False,32496.500000,35223.666667,37531.583333,...,31417.583333,32920.250000,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,48157.916667,5,non_working_day
57297,27789.500000,22,12,6,31,2017,False,29994.250000,32496.500000,35223.666667,...,31130.750000,31417.583333,32920.250000,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,5,non_working_day


In [121]:
# robust = SGDRegressor(alpha=.0001, loss='log', penalty='l2', max_iter=100, verbose=0, tol=0.001)
robust = PassiveAggressiveRegressor()

model = MultiOutputRegressor(robust)


In [122]:
multi_step_dat.iloc[47787:]

,value,hour,month,dayofweek,dayofmonth,year,holiday,value-1,value-2,value-3,...,n-740,n-741,n-742,n-743,n-744,n-745,n-746,n-747,season,working_day
48557,28344.166667,0,1,6,1,2017,False,28420.583333,30466.166667,32241.666667,...,33407.500000,32524.083333,33019.750000,33403.083333,34552.750000,34647.666667,35137.916667,39078.500000,6,non_working_day
48558,28367.916667,1,1,6,1,2017,False,28344.166667,28420.583333,30466.166667,...,37392.166667,33407.500000,32524.083333,33019.750000,33403.083333,34552.750000,34647.666667,35137.916667,6,non_working_day
48559,27524.500000,2,1,6,1,2017,False,28367.916667,28344.166667,28420.583333,...,44109.083333,37392.166667,33407.500000,32524.083333,33019.750000,33403.083333,34552.750000,34647.666667,6,non_working_day
48560,26112.250000,3,1,6,1,2017,False,27524.500000,28367.916667,28344.166667,...,46095.500000,44109.083333,37392.166667,33407.500000,32524.083333,33019.750000,33403.083333,34552.750000,6,non_working_day
48561,25143.500000,4,1,6,1,2017,False,26112.250000,27524.500000,28367.916667,...,45964.833333,46095.500000,44109.083333,37392.166667,33407.500000,32524.083333,33019.750000,33403.083333,6,non_working_day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57294,35223.666667,19,12,6,31,2017,False,37531.583333,38381.416667,36561.083333,...,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,48157.916667,49271.000000,49574.833333,5,non_working_day
57295,32496.500000,20,12,6,31,2017,False,35223.666667,37531.583333,38381.416667,...,32920.250000,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,48157.916667,49271.000000,5,non_working_day
57296,29994.250000,21,12,6,31,2017,False,32496.500000,35223.666667,37531.583333,...,31417.583333,32920.250000,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,48157.916667,5,non_working_day
57297,27789.500000,22,12,6,31,2017,False,29994.250000,32496.500000,35223.666667,...,31130.750000,31417.583333,32920.250000,33061.833333,34133.916667,37765.500000,42847.250000,45969.416667,5,non_working_day


In [131]:
X_train = multi_step_dat[multi_step_dat.year<2017].filter(regex='^(?!.*value|working_day|season).*$').values.astype(np.float32)
y_train = multi_step_dat[multi_step_dat.year<2017].filter(like='value').values

X_test = multi_step_dat[multi_step_dat.year>=2017].filter(regex='^(?!.*value|working_day|season).*$').values.astype(np.float32)
y_test = multi_step_dat[multi_step_dat.year>=2017].filter(like='value').values

scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  # apply same transformation to test data

model.partial_fit(X_train, y_train)

all_differences = []
for day in range(1,8760):
    X_train = multi_step_dat.iloc[(47787+((day-1)*1)):(47787+day*(1))].filter(regex='^(?!.*value|working_day|season).*$').values.astype(np.float32)
    y_train = multi_step_dat.iloc[(47787+((day-1)*1)):(47787+day*(1))].filter(like='value').values
    
    X_test = multi_step_dat.iloc[(47787+day*(1)):(47787+(day+1)*(1))].filter(regex='^(?!.*value|working_day|season).*$').values.astype(np.float32)
    y_test = multi_step_dat.iloc[(47787+day*(1)):(47787+(day+1)*(1))].filter(like='value').values
    
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
    
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)  # apply same transformation to test data

    model.partial_fit(X_train, y_train)

    predicted = model.predict(X_test)
    actual = y_test
#     if day >350:
#         plt.plot(range(24), actual[0])
#         plt.plot(range(24), predicted[0])
#         plt.show()
#         plt.close()
    
    difference = predicted - actual
    all_differences.append(difference)
    

ValueError: Found array with 0 sample(s) (shape=(0, 114)) while a minimum of 1 is required by StandardScaler.

In [ ]:

flattened_differences = np.concatenate(all_differences, axis=0)

differences_dataframe = pd.DataFrame({'differences':flattened_differences[0]})

multi_differences = differences_dataframe
multi_differences
sns.distplot(multi_differences.differences)
plt.axvline(x=multi_differences.differences.quantile(0.05), linestyle='--', linewidth=2.5, label="5% Percentile", c='b')
plt.axvline(x=multi_differences.differences.quantile(0.95), linestyle='--', linewidth=2.5, label="95% Percentile", c='purple')
plt.ylabel("frequency of occurence", labelpad=14)
plt.axvline(x=0, label="0", c='g')
plt.axvline(x=-6000, label="Max Tendered National Grid Reserve", c='r')
plt.axvline(x=6000, label="Max Tendered National Grid Reserve", c='r')
plt.axvline(x=-2000, label="Average Available Tendered National Grid Reserve", c='b')
plt.axvline(x=2000, label="Average Available Tendered National Grid Reserve", c='b')

plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left");